In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, date
import geopandas as gpd
from shapely.geometry import Point

In [2]:
df_acidentes_raw = pd.read_csv('data/Acidentes/sinistros_com_chuva_2022-2025.csv', delimiter=';')

### Limpeza dos dados

In [3]:
# Limpeza
df_acidentes_raw['latitude'] = df_acidentes_raw['latitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['longitude'] = df_acidentes_raw['longitude'].str.replace(',', '.').astype(float)
df_acidentes_raw['data_sinistro'] = pd.to_datetime(df_acidentes_raw['data_sinistro'], dayfirst=True)
df_acidentes_raw['hora_sinistro'] = df_acidentes_raw['hora_sinistro'].astype(str).replace('99:99', np.nan)
df_acidentes_raw['hora_sinistro_dt'] = pd.to_datetime(df_acidentes_raw['hora_sinistro'], format='%H:%M')


# Mapear 'tipo_via' para números (0: Municipal, 1: Rodovia)
tipo_via_map = {'VIAS MUNICIPAIS': 0, 'RODOVIAS': 1}
df_acidentes_raw['tipo_via_num'] = df_acidentes_raw['tipo_via'].map(tipo_via_map)

# Mapear 'Chuva' para binário (0: Sem chuva, 1: Com chuva)
df_acidentes_raw['Chuva'] = df_acidentes_raw['Chuva'].apply(lambda x: 1 if x != 'Sem chuva' else 0)

# Extrair data e hora
df_acidentes_raw['hora'] = df_acidentes_raw['hora_sinistro_dt'].dt.hour
df_acidentes_raw.fillna({'hora': -1}, inplace=True)
df_acidentes_raw['hora'] = df_acidentes_raw['hora'].astype(int)
df_acidentes_raw['data'] = df_acidentes_raw['data_sinistro'].dt.date
df_acidentes_raw['dia_semana'] = df_acidentes_raw['data_sinistro'].dt.dayofweek
df_acidentes_raw['mes'] = df_acidentes_raw['data_sinistro'].dt.month
df_acidentes_raw['is_weekend'] = df_acidentes_raw['dia_semana'].isin([5, 6]).astype(int)

# Definir a classe positiva
df_acidentes_raw['Sinistro'] = 1

colunas_veiculos = [col for col in df_acidentes_raw.columns if 'tp_veiculo' in col]
colunas_base = ['latitude', 'longitude', 'data', 'dia_semana', 'mes', 'is_weekend', 'hora', 'Sinistro', 'Chuva', 'tipo_via_num']
df_positivos = df_acidentes_raw[colunas_base + colunas_veiculos]

print(f"Total de positivos: {len(df_positivos)}")
df_positivos

Total de positivos: 8189


,latitude,longitude,data,dia_semana,mes,is_weekend,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,2022-01-01,5,1,1,4,1,0,0.0,0,0,0,0,0,0,0
1,-22.305238,-49.097946,2022-01-01,5,1,1,4,1,0,0.0,0,0,0,0,0,0,0
2,-22.306460,-49.104453,2022-01-01,5,1,1,4,1,0,0.0,0,0,1,0,0,0,0
3,-22.312349,-49.122265,2022-01-02,6,1,1,19,1,0,0.0,0,0,0,0,0,0,0
4,-22.338166,-49.053244,2022-01-02,6,1,1,20,1,0,0.0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,-22.391376,-49.069106,2025-02-28,4,2,0,22,1,0,0.0,0,0,0,0,0,0,3
8185,-22.295708,-49.045586,2025-02-28,4,2,0,13,1,0,0.0,0,0,1,0,0,0,1
8186,-22.338551,-49.069617,2025-02-28,4,2,0,13,1,0,0.0,0,0,0,0,0,0,2
8187,-22.305703,-49.051948,2025-02-28,4,2,0,18,1,0,0.0,0,0,0,0,0,0,1


### Função temporal

In [4]:
# Criamos o conjunto de busca para checagem de colisão (muito rápido)
lookup_acidentes = set(zip(
    df_positivos['latitude'],
    df_positivos['longitude'],
    df_positivos['data'], # A coluna 'data' já é um objeto date
    df_positivos['hora']
))

# Pega a lista de colunas de veículos para usar depois
colunas_veiculos = [col for col in df_positivos.columns if 'tp_veiculo' in col]

def gerar_features_temporais_aleatorias():
    start_date = date(2022, 1, 1)
    end_date = date(2025, 12, 31)
    total_days = (end_date - start_date).days
    
    data_aleatoria = start_date + timedelta(days=random.randint(0, total_days))
    hora_aleatoria = random.randint(0, 23)
    dia_semana_aleatorio = data_aleatoria.weekday() # Segunda=0, Domingo=6
    
    features = {
        'data': data_aleatoria,
        'hora': hora_aleatoria,
        'dia_semana': dia_semana_aleatorio,
        'mes': data_aleatoria.month,
        'is_weekend': 1 if dia_semana_aleatorio >= 5 else 0
    }
    return features

def sortear_chuva(probabilidade_chuva=0.2):
    return 1 if random.random() < probabilidade_chuva else 0

def sortear_veiculos_aleatorios(colunas_veiculos, prob_sem_veiculo=0.10):
    """
    Sorteia tipos de veículos para uma amostra negativa, simulando tráfego normal.
    
    Argumentos:
    colunas_veiculos -- A lista de todas as colunas 'tp_veiculo_*'
    prob_sem_veiculo -- A probabilidade de a amostra não ter nenhum veículo (ex: madrugada)
    """
    
    # 10% de chance de ser um ponto sem veículo algum (todos 0)
    if random.random() < prob_sem_veiculo:
        return {col: 0 for col in colunas_veiculos}

    # Se tiver veículo, inicializa todos como 0
    veiculos_sorteados = {col: 0 for col in colunas_veiculos}

    # Ponderação de quais veículos são mais comuns no tráfego normal
    # Ajuste esses pesos conforme a realidade da sua cidade
    pesos = {
        'tp_veiculo_automovel': 0.70,
        'tp_veiculo_motocicleta': 0.20,
        'tp_veiculo_caminhao': 0.04,
        'tp_veiculo_onibus': 0.03,
        'tp_veiculo_bicicleta': 0.02,
        'tp_veiculo_outros': 0.01,
        'tp_veiculo_nao_disponivel': 0.0  # Não queremos sortear "não disponível"
    }

    # Filtra apenas as colunas que existem na lista e têm peso
    colunas_com_peso = [col for col in colunas_veiculos if col in pesos and pesos[col] > 0]
    lista_pesos = [pesos[col] for col in colunas_com_peso]

    if not colunas_com_peso:
        # Fallback caso os nomes das colunas não batam
        colunas_com_peso = [col for col in colunas_veiculos if 'nao_disponivel' not in col]
        if not colunas_com_peso:
            colunas_com_peso = [colunas_veiculos[0]] # Evitar erro
        lista_pesos = [1] * len(colunas_com_peso) # Peso igual

    # Normaliza os pesos para a soma ser 1
    soma_pesos = sum(lista_pesos)
    pesos_normalizados = [p / soma_pesos for p in lista_pesos]

    # Sorteia UMA coluna de veículo com base nos pesos
    col_sorteada = random.choices(colunas_com_peso, weights=pesos_normalizados, k=1)[0]
    
    # Define o veículo sorteado como 1 (simulando 1 veículo)
    # Poderia ser random.randint(1, 2) se quiséssemos simular mais
    veiculos_sorteados[col_sorteada] = 1
    
    return veiculos_sorteados

# Na célula [39]

def sortear_veiculos_aleatorios(colunas_veiculos, prob_sem_veiculo=0.00):
    """
    Sorteia tipos de veículos para uma amostra negativa, simulando tráfego normal.
    
    Argumentos:
    colunas_veiculos -- A lista de todas as colunas 'tp_veiculo_*'
    prob_sem_veiculo -- A probabilidade de a amostra não ter nenhum veículo (ex: madrugada)
    """
    
    # 10% de chance de ser um ponto sem veículo algum (todos 0)
    if random.random() < prob_sem_veiculo:
        return {col: 0 for col in colunas_veiculos}

    # Se tiver veículo, inicializa todos como 0
    veiculos_sorteados = {col: 0 for col in colunas_veiculos}

    # Ponderação de quais veículos são mais comuns no tráfego normal
    # Ajuste esses pesos conforme a realidade da sua cidade
    pesos = {
        'tp_veiculo_automovel': 0.80,
        'tp_veiculo_motocicleta': 0.10,
        'tp_veiculo_caminhao': 0.04,
        'tp_veiculo_onibus': 0.05,
        'tp_veiculo_bicicleta': 0.02,
        'tp_veiculo_outros': 0.01,
        'tp_veiculo_nao_disponivel': 0.0  # Não queremos sortear "não disponível"
    }

    # Filtra apenas as colunas que existem na lista e têm peso
    colunas_com_peso = [col for col in colunas_veiculos if col in pesos and pesos[col] > 0]
    lista_pesos = [pesos[col] for col in colunas_com_peso]

    if not colunas_com_peso:
        # Fallback caso os nomes das colunas não batam
        colunas_com_peso = [col for col in colunas_veiculos if 'nao_disponivel' not in col]
        if not colunas_com_peso:
            colunas_com_peso = [colunas_veiculos[0]] # Evitar erro
        lista_pesos = [1] * len(colunas_com_peso) # Peso igual

    # Normaliza os pesos para a soma ser 1
    soma_pesos = sum(lista_pesos)
    pesos_normalizados = [p / soma_pesos for p in lista_pesos]

    # Sorteia UMA coluna de veículo com base nos pesos
    col_sorteada = random.choices(colunas_com_peso, weights=pesos_normalizados, k=1)[0]
    
    # Define o veículo sorteado como 1 (simulando 1 veículo)
    # Poderia ser random.randint(1, 2) se quiséssemos simular mais
    veiculos_sorteados[col_sorteada] = 1
    
    return veiculos_sorteados

In [5]:
# 1. Calcular a soma de cada tipo de veículo nos acidentes
total_acidentes = len(df_positivos)
veiculo_soma = df_positivos[colunas_veiculos].sum()

# 2. Calcular a probabilidade (distribuição) de cada um
# (Usamos .sum() na soma caso um acidente tenha mais de um veículo)
veiculo_probs = veiculo_soma / veiculo_soma.sum()

# 3. Preparar listas para a amostragem aleatória
veiculo_nomes = veiculo_probs.index.tolist()
veiculo_prob_valores = veiculo_probs.values

print("Distribuição de veículos inferida dos acidentes (será usada para amostras negativas):")
print(veiculo_probs)

Distribuição de veículos inferida dos acidentes (será usada para amostras negativas):
tp_veiculo_bicicleta         0.014908
tp_veiculo_caminhao          0.023752
tp_veiculo_motocicleta       0.365256
tp_veiculo_nao_disponivel    0.058497
tp_veiculo_onibus            0.016298
tp_veiculo_outros            0.019078
tp_veiculo_automovel         0.502211
dtype: float64


### Geração dos Negativos Tipo 1 (Mesmo Local; Data, Hora e Chuva Aleatório)

In [6]:
negativos_tipo1_lista = []
prop_negativos_tipo1 = 50

print("Gerando Negativos Tipo 1 (Contraste Temporal)...")
for index, acidente in df_positivos.iterrows():
    lat = acidente['latitude']
    lon = acidente['longitude']

    for _ in range(prop_negativos_tipo1):
    
        # Loop para garantir que o novo ponto sorteado não seja um acidente real
        while True:
            features_temporais = gerar_features_temporais_aleatorias()
            if (lat, lon, features_temporais['data'], features_temporais['hora']) not in lookup_acidentes:
                break
                
        # Cria o dicionário do novo negativo com todas as colunas
        novo_negativo = {
            'latitude': lat,
            'longitude': lon,
            'tipo_via_num': acidente['tipo_via_num'], # Mantém o tipo de via do local
            'Chuva': sortear_chuva(),
            'Sinistro': 0
        }
        
        # Adiciona as novas features temporais
        novo_negativo.update(features_temporais)
        
        # Sorteia os veículos para a amostra negativa (simulando tráfego)
        veiculos_sorteados = sortear_veiculos_aleatorios(colunas_veiculos)
        novo_negativo.update(veiculos_sorteados)
            
        negativos_tipo1_lista.append(novo_negativo)

df_negativos_tipo1 = pd.DataFrame(negativos_tipo1_lista)
print("\nNegativos Tipo 1 Gerados com Sucesso.")
print(f"Total de negativos tipo 1: {len(df_negativos_tipo1)}")
df_negativos_tipo1

Gerando Negativos Tipo 1 (Contraste Temporal)...

Negativos Tipo 1 Gerados com Sucesso.
Total de negativos tipo 1: 409450


,latitude,longitude,tipo_via_num,Chuva,Sinistro,data,hora,dia_semana,mes,is_weekend,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.318170,-49.114148,0.0,0,0,2025-10-30,19,3,10,0,0,0,0,0,0,0,1
1,-22.318170,-49.114148,0.0,0,0,2023-02-26,2,6,2,1,0,0,0,0,0,0,1
2,-22.318170,-49.114148,0.0,1,0,2024-12-04,22,2,12,0,0,0,0,0,0,0,1
3,-22.318170,-49.114148,0.0,0,0,2023-02-17,20,4,2,0,0,0,0,0,0,0,1
4,-22.318170,-49.114148,0.0,0,0,2022-06-07,13,1,6,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409445,-22.310847,-49.021009,0.0,0,0,2024-07-29,12,0,7,0,0,0,0,0,0,0,1
409446,-22.310847,-49.021009,0.0,0,0,2024-04-23,6,1,4,0,0,0,0,0,0,0,1
409447,-22.310847,-49.021009,0.0,0,0,2022-05-22,2,6,5,1,0,0,0,0,0,0,1
409448,-22.310847,-49.021009,0.0,0,0,2024-07-10,23,2,7,0,0,0,0,0,0,0,1


### Geração dos Negativos Tipo 2 (Local Aleatório; Data, Hora e tempo Aleatório)

In [7]:
gdf_ruas = gpd.read_file('ruas_de_bauru.gpkg')

# Mapeia os tipos de via do OpenStreetMap para o padrão (0 ou 1)
highway_map = {
     'motorway': 1, 'trunk': 1, 'primary': 0, 'secondary': 0,
     'tertiary': 0, 'residential': 0, 'unclassified': 0
}

gdf_ruas['tipo_via_num'] = gdf_ruas['highway'].map(highway_map).fillna(0).astype(int)

min_lon, min_lat, max_lon, max_lat = gdf_ruas.total_bounds
uniao_ruas = gdf_ruas.union_all().buffer(0.0001)

negativos_tipo2_lista = []
prop_negativos_tipo2 = len(df_positivos) * 50

print("\nGerando Negativos Tipo 2 (Contraste Espacial)...")
while len(negativos_tipo2_lista) < prop_negativos_tipo2:
    lon_aleatorio = random.uniform(min_lon, max_lon)
    lat_aleatoria = random.uniform(min_lat, max_lat)
    ponto_aleatorio = Point(lon_aleatorio, lat_aleatoria)
    
    if ponto_aleatorio.within(uniao_ruas):
        indices_proximos = gdf_ruas.sindex.nearest(ponto_aleatorio)
        indice_da_rua = indices_proximos[1][0] # Extrai o número inteiro
        rua_mais_proxima = gdf_ruas.iloc[[indice_da_rua]]
        
        tipo_via_ponto = rua_mais_proxima['tipo_via_num'].iloc[0]
        
        features_temporais = gerar_features_temporais_aleatorias()
        
        novo_negativo = {
            'latitude': lat_aleatoria,
            'longitude': lon_aleatorio,
            'tipo_via_num': tipo_via_ponto,
            'Chuva': sortear_chuva(),
            'Sinistro': 0
        }

        novo_negativo.update(features_temporais)
        veiculos_sorteados = sortear_veiculos_aleatorios(colunas_veiculos)
        novo_negativo.update(veiculos_sorteados)
            
        negativos_tipo2_lista.append(novo_negativo)

df_negativos_tipo2 = pd.DataFrame(negativos_tipo2_lista)
print(f"Gerado {len(negativos_tipo2_lista)}/{prop_negativos_tipo2} negativos tipo 2...")
print("\nNegativos Tipo 2 Gerados com Sucesso.")
df_negativos_tipo2


Gerando Negativos Tipo 2 (Contraste Espacial)...
Gerado 409450/409450 negativos tipo 2...

Negativos Tipo 2 Gerados com Sucesso.


,latitude,longitude,tipo_via_num,Chuva,Sinistro,data,hora,dia_semana,mes,is_weekend,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.338161,-49.086674,0,0,0,2025-07-22,13,1,7,0,0,0,1,0,0,0,0
1,-22.328324,-49.104317,0,0,0,2023-04-27,18,3,4,0,0,0,0,0,0,0,1
2,-22.300194,-49.080755,0,0,0,2025-07-22,21,1,7,0,0,0,1,0,0,0,0
3,-22.343592,-49.058126,0,0,0,2022-08-15,22,0,8,0,0,1,0,0,0,0,0
4,-22.178710,-49.223467,1,0,0,2024-02-22,8,3,2,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409445,-22.295047,-49.063763,0,0,0,2025-08-21,8,3,8,0,0,1,0,0,0,0,0
409446,-22.339721,-49.089007,0,1,0,2024-12-28,10,5,12,1,0,0,0,0,0,0,1
409447,-22.321512,-49.060662,0,1,0,2023-03-02,15,3,3,0,0,0,0,0,0,0,1
409448,-22.312979,-49.090445,0,0,0,2025-05-10,2,5,5,1,0,0,0,0,0,0,1


### Unir, embaralhar e salvar

In [8]:
df_final = pd.concat([df_positivos, df_negativos_tipo1], ignore_index=True)
df_final = pd.concat([df_final, df_negativos_tipo2], ignore_index=True)

# Embaralha o dataset final para garantir aleatoriedade
df_final = df_final.sample(frac=1).reset_index(drop=True)

df_final.to_csv('dataset_final_para_modelo_1_100.csv', index=False, decimal=',')

print(f"Total de amostras: {len(df_final)}")
print("Distribuição das classes:")
print(df_final['Sinistro'].value_counts())
df_final

Total de amostras: 827089
Distribuição das classes:
Sinistro
0    818900
1      8189
Name: count, dtype: int64


,latitude,longitude,data,dia_semana,mes,is_weekend,hora,Sinistro,Chuva,tipo_via_num,tp_veiculo_bicicleta,tp_veiculo_caminhao,tp_veiculo_motocicleta,tp_veiculo_nao_disponivel,tp_veiculo_onibus,tp_veiculo_outros,tp_veiculo_automovel
0,-22.323152,-49.066030,2022-10-24,0,10,0,23,0,0,0.0,0,0,0,0,0,0,1
1,-22.330302,-49.106223,2023-03-02,3,3,0,12,0,0,0.0,0,0,0,0,0,0,1
2,-22.335932,-49.067196,2025-01-08,2,1,0,17,0,0,0.0,0,0,0,0,0,0,1
3,-22.327175,-49.116669,2024-01-23,1,1,0,19,0,0,0.0,0,0,0,0,0,0,1
4,-22.330951,-49.088352,2022-09-19,0,9,0,20,0,0,0.0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827084,-22.309319,-49.073527,2025-01-16,3,1,0,15,0,0,0.0,0,0,1,0,0,0,0
827085,-22.325896,-49.079704,2024-05-17,4,5,0,9,0,0,0.0,0,0,0,0,0,0,1
827086,-22.351893,-49.032062,2024-02-01,3,2,0,13,0,0,0.0,0,0,0,0,0,0,1
827087,-22.291783,-49.071040,2022-07-10,6,7,1,9,0,0,0.0,0,0,0,0,0,0,1
